In [95]:
# __author__ = "Philipp Kats"
# __date__ = "2015_11_05"
%pylab inline
import pandas as pd
import geopandas as gp
import numpy as np

from geopandas.tools import sjoin
from shapely.geometry import Point

import os


PARQA = os.getenv('PARQA') #basic path

Populating the interactive namespace from numpy and matplotlib


In [77]:
calls = pd.read_csv(PARQA + 'data/311/311DPR.csv', encoding='utf8', na_values='Unspecified')
# calls = calls.rename(columns={'Longitude':'lon','Latitude':'lat'})

In [78]:
calls.shape

(82590, 54)

In [79]:
calls.columns

Index([                    u'Unnamed: 0',                     u'Unique Key',
                         u'Created Date',                    u'Closed Date',
                               u'Agency',                    u'Agency Name',
                       u'Complaint Type',                     u'Descriptor',
                        u'Location Type',                   u'Incident Zip',
                     u'Incident Address',                    u'Street Name',
                       u'Cross Street 1',                 u'Cross Street 2',
                u'Intersection Street 1',          u'Intersection Street 2',
                         u'Address Type',                           u'City',
                             u'Landmark',                  u'Facility Type',
                               u'Status',                       u'Due Date',
               u'Resolution Description', u'Resolution Action Updated Date',
                      u'Community Board',                        u'Borough',

In [80]:
myCalls = calls[['Park Facility Name','Descriptor','Created Date','Closed Date','Longitude','Latitude','Location Type']]
myCalls['Park Facility Name'] = myCalls['Park Facility Name'].str.lower()
myCalls['Park Facility Name'].head()

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


0                                    NaN
1    geo soilan park - battery park city
2                        brookville park
3                                    NaN
4                                    NaN
Name: Park Facility Name, dtype: object

In [81]:
myCalls[pd.isnull(myCalls['Park Facility Name'])]

,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type
0,NaN,Snow or Ice,12/31/2010 09:04:48 PM,01/03/2011 12:03:59 PM,-73.931120,40.668798,Park
3,NaN,Snow or Ice,12/31/2010 03:36:37 PM,01/03/2011 09:41:24 AM,-73.962835,40.688556,Park
4,NaN,Snow or Ice,12/31/2010 03:03:16 PM,01/03/2011 12:15:38 PM,-73.999809,40.636935,Park
6,NaN,Snow or Ice,12/31/2010 12:59:59 PM,01/03/2011 12:23:04 PM,-73.999456,40.609951,Park
7,NaN,Snow or Ice,12/31/2010 12:12:02 PM,01/03/2011 12:19:51 PM,-73.977616,40.633153,Park
8,NaN,Snow or Ice,12/31/2010 12:08:23 PM,01/03/2011 02:33:24 PM,-73.962486,40.705351,Park
9,NaN,Dead Animal,12/31/2010 11:27:06 AM,01/04/2011 11:02:35 AM,-73.983284,40.672797,Park
16,NaN,Snow or Ice,12/31/2010 09:04:23 AM,01/06/2011 03:04:46 PM,-73.870572,40.733116,Park
23,NaN,Snow or Ice,12/30/2010 09:35:00 PM,01/03/2011 01:34:56 PM,-74.103509,40.616415,Park
25,NaN,Snow or Ice,12/30/2010 09:20:45 PM,01/03/2011 02:35:28 PM,-73.960783,40.706465,Park


In [82]:
## need to check the closest park and if distance is small, attach to it. 
##for now - using spatial joint to match to Disricts
print 100.0*len(myCalls[pd.isnull(myCalls['Park Facility Name'])])/len(myCalls), '%'

33.9060418937 %


In [83]:
notNamedCalls = myCalls[pd.isnull(myCalls['Park Facility Name'])]
namedCalls = myCalls[pd.notnull(myCalls['Park Facility Name'])]
print notNamedCalls.shape
print namedCalls.shape

(28003, 7)
(54587, 7)


## I. Ontology

In [85]:
onto = pd.read_csv(PARQA + 'parqa/311/ONTOLOGY/ontology/Ontology_matched.csv', index_col=0)

In [86]:
onto.head(2)

,cleanName,NAME,Type,valid,parkDistrict,lat,lon
0,geo soilan park - battery park city,battery park city,other,av,M-01,-74.016893,40.711882
1,geo soilan park - battery park city,battery park city,other,av,M-01,-74.016890,40.712719


In [84]:
ncMatched = namedCalls.merge(onto, how='left',left_on='Park Facility Name', right_on='cleanName')
ncMatched.head(1)

,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type,cleanName,NAME,Type,valid,parkDistrict,lat,lon
0,geo soilan park - battery park city,Graffiti or Vandalism,12/31/2010 04:31:52 PM,12/31/2010 05:36:58 PM,NaN,NaN,Park,geo soilan park - battery park city,battery park city,other,av,M-01,-74.016893,40.711882


In [108]:
# ncMatched[pd.isnull(ncMatched.NAME)] # those i coldn't recognize

## II. Geolocated

In [118]:
pDistricts = gp.read_file(PARQA + 'data/SHP/Park_Districts/ParkDistrict.shp')
pD = pDistricts[['SYSTEM','geometry']]

In [119]:
def toGeoDataFrame(df, lat='Latitude',lon='Longitude'):
    '''dataframe to geodataframe'''
    df['geometry'] = df.apply(lambda z: Point(z[lon], z[lat]), axis=1)
    df = gp.GeoDataFrame(df)
    df.crs = {'init': 'epsg:4326', 'no_defs': True}
    return df 

In [120]:
nn = toGeoDataFrame(notNamedCalls[pd.notnull(notNamedCalls.Longitude)])

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [121]:
# nn.plot()

In [122]:
nn = nn.to_crs(pD.crs)
nn = sjoin(nn, pD, how="left").rename(columns={'SYSTEM':'parkDistrict'})
# nn.head(2)

In [107]:
nn.head()

,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type,geometry,index_right,parkDistrict
0,NaN,Snow or Ice,12/31/2010 09:04:48 PM,01/03/2011 12:03:59 PM,-73.931120,40.668798,Park,POINT (1003357.999975349 182945.0143717242),29,B-09
3,NaN,Snow or Ice,12/31/2010 03:36:37 PM,01/03/2011 09:41:24 AM,-73.962835,40.688556,Park,POINT (994556.9999903567 190138.014400504),22,B-02
4,NaN,Snow or Ice,12/31/2010 03:03:16 PM,01/03/2011 12:15:38 PM,-73.999809,40.636935,Park,POINT (984302.9999955755 171329.0143867982),32,B-12
6,NaN,Snow or Ice,12/31/2010 12:59:59 PM,01/03/2011 12:23:04 PM,-73.999456,40.609951,Park,POINT (984401.0000036418 161498.0143712223),31,B-11
7,NaN,Snow or Ice,12/31/2010 12:12:02 PM,01/03/2011 12:19:51 PM,-73.977616,40.633153,Park,POINT (990463.0000056089 169952.0143654562),34,B-14


In [123]:
def gdfToCsv(p):    
    p['lat'] = p.geometry.apply(lambda x: x.coords[0][0])
    p['lon'] = p.geometry.apply(lambda x: x.coords[0][1])
    return p.drop('geometry',1)

In [124]:
nn = gdfToCsv(nn)

In [125]:
# nn.drop(['index_right'], axis=1, inplace=1)
# nn.drop(['Latitude','Longitude'], axis=1, inplace=1)
nn.columns

Index([u'Park Facility Name',         u'Descriptor',       u'Created Date',
              u'Closed Date',          u'Longitude',           u'Latitude',
            u'Location Type',        u'index_right',       u'parkDistrict',
                      u'lat',                u'lon'],
      dtype='object')

In [110]:
ncMatched.columns

Index([u'Park Facility Name',         u'Descriptor',       u'Created Date',
              u'Closed Date',          u'Longitude',           u'Latitude',
            u'Location Type',          u'cleanName',               u'NAME',
                     u'Type',              u'valid',       u'parkDistrict',
                      u'lat',                u'lon'],
      dtype='object')

In [126]:
MatchedCalls = pd.concat([nn,ncMatched])

In [127]:
MatchedCalls.shape

(88961, 15)

In [129]:
MatchedCalls.head(4)

,Closed Date,Created Date,Descriptor,Latitude,Location Type,Longitude,NAME,Park Facility Name,Type,cleanName,index_right,lat,lon,parkDistrict,valid
0,01/03/2011 12:03:59 PM,12/31/2010 09:04:48 PM,Snow or Ice,40.668798,Park,-73.931120,NaN,NaN,NaN,NaN,29,1003357.999975,182945.014372,B-09,NaN
3,01/03/2011 09:41:24 AM,12/31/2010 03:36:37 PM,Snow or Ice,40.688556,Park,-73.962835,NaN,NaN,NaN,NaN,22,994556.999990,190138.014401,B-02,NaN
4,01/03/2011 12:15:38 PM,12/31/2010 03:03:16 PM,Snow or Ice,40.636935,Park,-73.999809,NaN,NaN,NaN,NaN,32,984302.999996,171329.014387,B-12,NaN
6,01/03/2011 12:23:04 PM,12/31/2010 12:59:59 PM,Snow or Ice,40.609951,Park,-73.999456,NaN,NaN,NaN,NaN,31,984401.000004,161498.014371,B-11,NaN


In [131]:
MatchedCalls[['Closed Date',
              'Created Date',
              'Location Type',
              'NAME',
              'lat',
              'lon',
              'parkDistrict']].to_csv(PARQA + 'data/311/MatchedCalls.cav')